This notebook is a continuation of the initial Heart notebook.
It is going to cover the deep learning model using sigmoid as the activation function in the output layer.

In [1]:
import io
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from sklearn.model_selection import GridSearchCV, train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
training_path = '/content/drive/MyDrive/Colab Notebooks/Ind_Proj/deeplearning1_training.csv'
traininglabel_path = '/content/drive/MyDrive/Colab Notebooks/Ind_Proj/deeplearning1_traininglabel.csv'
eval_path = '/content/drive/MyDrive/Colab Notebooks/Ind_Proj/deeplearning1_evaluate.csv'
eval_label_path = '/content/drive/MyDrive/Colab Notebooks/Ind_Proj/deeplearning1_evaluatelabel.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/Ind_Proj/deeplearning1_test.csv'
testlabels_path = '/content/drive/MyDrive/Colab Notebooks/Ind_Proj/deeplearning1_testlabels.csv'
main_data_path = '/content/drive/MyDrive/Colab Notebooks/Ind_Proj/deeplearning1.csv'

deeplearning1_training = pd.read_csv(training_path)
deeplearning1_traininglabel = pd.read_csv(traininglabel_path)
deeplearning1_evaluate = pd.read_csv(eval_path)
deeplearning1_evaluatelabel = pd.read_csv(eval_label_path)
deeplearning1_test = pd.read_csv(test_path)
deeplearning1_testlabels = pd.read_csv(testlabels_path)
main_data = pd.read_csv(main_data_path)

In [3]:
print("Training shape:", deeplearning1_training.shape)
print("Evaluation shape:", deeplearning1_evaluate.shape)
print("Test shape:", deeplearning1_test.shape)
print("Main data shape: ", main_data.shape)
#no need to reshape or normalize

Training shape: (700, 11)
Evaluation shape: (109, 11)
Test shape: (109, 11)
Main data shape:  (918, 12)


In [4]:
deeplearning1_testlabels.head()

,HeartDisease
0,Normal
1,Normal
2,Normal
3,Normal
4,Normal


In [5]:
label_encoder = LabelEncoder()

deeplearning1_traininglabel = label_encoder.fit_transform(deeplearning1_traininglabel.iloc[:, 0])
deeplearning1_evaluatelabel = label_encoder.fit_transform(deeplearning1_evaluatelabel.iloc[:, 0])
deeplearning1_testlabels = label_encoder.fit_transform(deeplearning1_testlabels.iloc[:, 0])

In [6]:
#Verify
deeplearning1_traininglabel.shape

(700,)

In [7]:
#Normal: 1 Heart Disease: 0
deeplearning1_testlabels

array([1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1])

In [8]:
#edit or change the datasets.
main_data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,0.244898,1,0.333333,0.70,0.479270,0,0.5,0.788732,0,0.295455,1.0,Normal
1,0.428571,0,0.666667,0.80,0.298507,0,0.5,0.676056,0,0.409091,0.5,Heart Disease
2,0.183673,1,0.333333,0.65,0.469320,0,1.0,0.267606,0,0.295455,1.0,Normal
3,0.408163,0,0.000000,0.69,0.354892,0,0.5,0.338028,1,0.465909,0.5,Heart Disease
4,0.530612,1,0.666667,0.75,0.323383,0,0.5,0.436620,0,0.295455,1.0,Normal


In [9]:
#split the main data part randomly into training, training label, evaluation, evaluation labels, test and test labels
#make sure that the training dataset has equal amounts of Normal and Heart Disease
#randomize the data
main_data = main_data.sample(frac=1, random_state=22)#.reset_index(drop=True)
training = main_data.iloc[:700]
remaining = main_data.iloc[700:]
evaluation = remaining.iloc[:109]
test = remaining.iloc[109:]
deeplearning1_training = training.iloc[:, :-1]
deeplearning1_traininglabel = training.iloc[:,-1]
deeplearning1_evaluate = evaluation.iloc[:, :-1]
deeplearning1_evaluatelabel = evaluation.iloc[:,-1]
deeplearning1_test = test.iloc[:, :-1]
deeplearning1_testlabels = test.iloc[:,-1]

In [10]:
#make sure the number of Normal = number of Heart Disease, might reduce the number of records for training.
#current number: 700
#seems like a nice proportion for both sides so i'll leave it like this for now.
normal_train = deeplearning1_traininglabel[deeplearning1_traininglabel=='Normal']
heart_train = deeplearning1_traininglabel[deeplearning1_traininglabel=='Heart Disease']

In [11]:
print(heart_train.shape)
print(normal_train.shape)

(383,)
(317,)


In [12]:
deeplearning1_test.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
289,0.163265,1,0.333333,0.60,0.275290,0,0.5,0.845070,0,0.295455,1.0
850,0.693878,0,0.000000,0.80,0.271973,0,0.0,0.598592,0,1.000000,0.0
711,0.775510,1,0.000000,0.60,0.500829,0,0.0,0.640845,0,0.340909,0.5
179,0.367347,0,0.000000,0.65,0.394693,0,0.5,0.211268,0,0.295455,1.0
63,0.367347,1,0.000000,0.60,0.459370,0,0.5,0.457746,1,0.409091,0.5


In [13]:
deeplearning1_testlabels.head()

289           Normal
850    Heart Disease
711           Normal
179           Normal
63     Heart Disease
Name: HeartDisease, dtype: object

In [14]:
label_encoder = LabelEncoder()
'''
deeplearning1_traininglabel = label_encoder.fit_transform(deeplearning1_traininglabel.iloc[:])
deeplearning1_evaluatelabel = label_encoder.fit_transform(deeplearning1_evaluatelabel.iloc[:, 0])
deeplearning1_testlabels = label_encoder.fit_transform(deeplearning1_testlabels.iloc[:, 0])
label_encoder = LabelEncoder()
'''
deeplearning1_traininglabel = label_encoder.fit_transform(deeplearning1_traininglabel.values)
deeplearning1_evaluatelabel = label_encoder.fit_transform(deeplearning1_evaluatelabel.values)
deeplearning1_testlabels = label_encoder.fit_transform(deeplearning1_testlabels.values)

In [15]:
deeplearning1_testlabels

array([1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1])

In [16]:
model = keras.Sequential()

In [17]:
#add the hidden layers
#we'll start with two hidden layers, one with 512 neurons and the second as an output layer with 2 neurons
model.add(layers.Dense(700, activation="relu"))
model.add(layers.Dense(400, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

In [66]:
#compile the model
opt = keras.optimizers.RMSprop(learning_rate=0.001)
#opt = keras.optimizers.Adam(learning_rate=0.001)

In [67]:
model.compile(
    optimizer=opt,
    #loss="sparse_categorical_crossentropy",
    loss = "binary_crossentropy",
    metrics=["accuracy"]
)

In [68]:
model.fit(deeplearning1_training, deeplearning1_traininglabel, epochs = 30, batch_size = 64)

Epoch 1/30
11/11 [==============================] - 1s 7ms/step - loss: 0.1240 - accuracy: 0.9757
Epoch 2/30
11/11 [==============================] - 0s 8ms/step - loss: 0.0495 - accuracy: 0.9814
Epoch 3/30
11/11 [==============================] - 0s 7ms/step - loss: 0.0441 - accuracy: 0.9829
Epoch 4/30
11/11 [==============================] - 0s 9ms/step - loss: 0.0542 - accuracy: 0.9800
Epoch 5/30
11/11 [==============================] - 0s 8ms/step - loss: 0.0431 - accuracy: 0.9871
Epoch 6/30
11/11 [==============================] - 0s 7ms/step - loss: 0.0417 - accuracy: 0.9900
Epoch 7/30
11/11 [==============================] - 0s 8ms/step - loss: 0.0451 - accuracy: 0.9814
Epoch 8/30
11/11 [==============================] - 0s 7ms/step - loss: 0.0608 - accuracy: 0.9786
Epoch 9/30
11/11 [==============================] - 0s 8ms/step - loss: 0.0242 - accuracy: 0.9914
Epoch 10/30
11/11 [==============================] - 0s 7ms/step - loss: 0.0404 - accuracy: 0.9871
Epoch 11/30
11/11 [

In [69]:
#save the weights
#save & load the weights
model.save_weights('/content/drive/MyDrive/Colab Notebooks/sigmoid weights')

In [70]:
model.load_weights('/content/drive/MyDrive/Colab Notebooks/sigmoid weights')

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/sigmoid model')

In [ ]:
#load the model
model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/sigmoid model')

In [71]:
#Evaluate model
test_loss, test_acc = model.evaluate(deeplearning1_evaluate, deeplearning1_evaluatelabel)

4/4 [==============================] - 0s 7ms/step - loss: 0.7765 - accuracy: 0.8716


In [72]:
print(f"Evaluation Loss: {test_loss:.3f}")
print(f"Evaluation accuracy: {test_acc:.3f}")

Evaluation Loss: 0.777
Evaluation accuracy: 0.872


In [73]:
#Predictions
test = deeplearning1_test[0:10]
predictions = model.predict(test)

1/1 [==============================] - 0s 90ms/step


In [74]:
print(predictions)

[[1.0000000e+00]
 [5.6714114e-25]
 [1.2393136e-03]
 [1.0000000e+00]
 [3.6678184e-04]
 [1.0000000e+00]
 [1.0000000e+00]
 [9.9996269e-01]
 [5.0301157e-04]
 [3.8602331e-14]]


In [75]:
#predict first 10 patient results
print("0: Heart Disease")
print("1: Normal")
print("-----------------------------------------------")
for x in range(10):
  print("                    ")
  print("Patient",x, "actual result: ", deeplearning1_testlabels[x])
  for y in range(predictions.shape[1]):
    print("Patient predicted result: ", y, ":", predictions[x, y])


0: Heart Disease
1: Normal
-----------------------------------------------
                    
Patient 0 actual result:  1
Patient predicted result:  0 : 1.0
                    
Patient 1 actual result:  0
Patient predicted result:  0 : 5.6714114e-25
                    
Patient 2 actual result:  1
Patient predicted result:  0 : 0.0012393136
                    
Patient 3 actual result:  1
Patient predicted result:  0 : 1.0
                    
Patient 4 actual result:  0
Patient predicted result:  0 : 0.00036678184
                    
Patient 5 actual result:  1
Patient predicted result:  0 : 1.0
                    
Patient 6 actual result:  1
Patient predicted result:  0 : 1.0
                    
Patient 7 actual result:  1
Patient predicted result:  0 : 0.9999627
                    
Patient 8 actual result:  0
Patient predicted result:  0 : 0.0005030116
                    
Patient 9 actual result:  0
Patient predicted result:  0 : 3.860233e-14


In [76]:
#Test accuracy
Test = deeplearning1_test
prediction = model.predict(Test)
#prediction = np.argmax(prediction, axis=1)


4/4 [==============================] - 0s 4ms/step


In [77]:
# if the probability is more than 0.5 (1) means it's normal
# and if it is less than 0.5 (0) means it's heart disease

prediction = prediction > 0.5
# Classification Report
#print(classification_report(val_image_gen.classes,prediction))

In [78]:
#classification report
print(classification_report(deeplearning1_testlabels, prediction))

              precision    recall  f1-score   support

           0       0.85      0.83      0.84        64
           1       0.77      0.80      0.78        45

    accuracy                           0.82       109
   macro avg       0.81      0.81      0.81       109
weighted avg       0.82      0.82      0.82       109



In [79]:
accuracy = accuracy_score(deeplearning1_testlabels, prediction)
print("Deep learning Accuracy:", accuracy)

Deep learning Accuracy: 0.8165137614678899


In [36]:
print("0: Heart Disease")
print("1: Normal")
print("-----------------------------------------------")
confuse_matrix2 = confusion_matrix(deeplearning1_testlabels, prediction)
print("Confusion Matrix:")
print(confuse_matrix2)
print("---------------------------------------------")
comp_table2 = pd.crosstab(deeplearning1_testlabels, prediction.flatten(), rownames=['Actual'], colnames=['Predicted'])
print(comp_table2)

0: Heart Disease
1: Normal
-----------------------------------------------
Confusion Matrix:
[[53 11]
 [ 9 36]]
---------------------------------------------
Predicted  False  True 
Actual                 
0             53     11
1              9     36


**Observation from Trial 5**:

The model is biased towards Heart Disease. Half more than half of the training set is Heart Disease so it's making it difficult for the model to be accurate when it needs to predict a Normal condition.

**Solution**: Go back to r studio and make the training dataset equal in terms of heart disease and normal conditions.

**Observation from Trial 9**:

The training accuracy seems to be on the same level as the others but the evaluation accuracy increased to 87%(that's the first time an accuracy of 80 or higher has been achieved from evaluation).

The test accuracy was higher (58.70%) than the previous trials (46.70%) but that is still relatively low.

**Observation from Trial 13**:

Training accuracy was 94%. I changed the model loss back to sparse_catergorical_crossentropy from binary_crossentropy and changed the activation from to softmax from sigmoid and increased the number of nuerons in the output layer.

The evaluation accuracy was similar to the rest but the test accuracy for prediction was 82% which is the highest so far compared to the other trials which had 58.70%

**Observation from Trial 20**:

The evaluation accuracy and testing accuracy are both good which means the model is finally improving.